In [ ]:
import os
os.chdir('/content/drive/My Drive/kaggle_study/weather contest')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data 준비

## Train data

In [ ]:
loc1 = pd.read_csv('plant1_loc1.csv')
loc2 = pd.read_csv('plant1_loc2.csv')
loc3 = pd.read_csv('plant1_loc3.csv')


In [ ]:
loc1.shape,loc2.shape,loc3.shape

((58749, 13), (58749, 13), (58749, 7))

In [ ]:
loc1.isnull().sum(),loc2.isnull().sum(),loc3.isnull().sum()

(datetime                      0
 plant1_train.tem_in_loc1      0
 plant1_train.hum_in_loc1      0
 plant1_train.tem_coil_loc1    0
 plant1_train.tem_out_loc1     0
 plant1_train.hum_out_loc1     0
 plant1_train.cond_loc1        0
 cond_with_swv                 0
 swv_value                     0
 24h_datetime                  0
 24h_cond                      0
 48h_datetime                  0
 48h_cond                      0
 dtype: int64, plant1_train.mea_ddhr         0
 plant1_train.hum_in_loc2      0
 plant1_train.tem_in_loc2      0
 plant1_train.tem_coil_loc2    0
 plant1_train.tem_out_loc1     0
 plant1_train.hum_out_loc1     0
 plant1_train.cond_loc2        0
 datetime                      0
 24h_datetime                  0
 48h_datetime                  0
 24h_cond                      0
 48h_cond                      0
 swv_value                     0
 dtype: int64, plant1_train.mea_ddhr         0
 plant1_train.tem_in_loc3      0
 plant1_train.hum_in_loc3      0
 plant1_train.t

In [ ]:
loc1.dropna(inplace=True)
loc2.dropna(inplace=True)
loc3.dropna(inplace=True)


### preprocessing

In [ ]:
from kaggle_study.KMA_Weather import preprocessing_24h48h

prep= preprocessing_24h48h

In [ ]:
plant= '1'
loc1_swv = prep.swv(loc1['plant'+plant+'_train.tem_in_loc1'],loc1['plant'+plant+'_train.tem_coil_loc1'],loc1['plant'+plant+'_train.hum_in_loc1'])
loc2_swv = prep.swv(loc2['plant'+plant+'_train.tem_in_loc2'],loc2['plant'+plant+'_train.tem_coil_loc2'],loc2['plant'+plant+'_train.hum_in_loc2'])
loc3_swv = prep.swv(loc3['plant'+plant+'_train.tem_in_loc3'],loc3['plant'+plant+'_train.tem_coil_loc3'],loc3['plant'+plant+'_train.hum_in_loc3'])

In [ ]:
loc1['swv_value'] = loc1_swv
loc2['swv_value'] = loc2_swv
loc3['swv_value'] = loc3_swv

In [ ]:
loc_num = 'loc1'
mms = MinMaxScaler()

scaled_loc1 = mms.fit_transform(loc1[['plant1_train.tem_in_'+loc_num, 'plant1_train.hum_in_'+loc_num,
       'plant1_train.tem_coil_'+loc_num, 'plant1_train.tem_out_loc1',
       'plant1_train.hum_out_loc1','swv_value']])

scaled_loc2 = mms.fit_transform(loc2[['plant1_train.tem_in_loc2', 'plant1_train.hum_in_loc2',
       'plant1_train.tem_coil_loc2', 'plant1_train.tem_out_loc1',
       'plant1_train.hum_out_loc1','swv_value']])

scaled_loc3 = mms.fit_transform(loc3[['plant1_train.tem_in_loc3', 'plant1_train.hum_in_loc3',
       'plant1_train.tem_coil_loc3', 'plant1_train.tem_out_loc1',
       'plant1_train.hum_out_loc1','swv_value']])

In [ ]:
x_train = np.concatenate([scaled_loc1,scaled_loc2,scaled_loc3])
y_train = np.concatenate([loc1['plant1_train.cond_loc1'],loc2['plant1_train.cond_loc2'],loc3['plant1_train.cond_loc3']])

In [ ]:
x_train.shape,y_train.shape

((175862, 6), (175862,))

In [ ]:
x_train

array([[0.53217932, 0.1754386 , 0.43819166, 0.43417528, 0.39491942,
        0.30537667],
       [0.48779405, 0.22556391, 0.41464563, 0.39590509, 0.57636888,
        0.27889323],
       [0.46560142, 0.28822055, 0.41464563, 0.37677   , 0.54434838,
        0.26601845],
       ...,
       [0.4187291 , 0.34719582, 0.42207634, 0.38442403, 0.40601985,
        0.29463786],
       [0.4180602 , 0.35586977, 0.42013129, 0.38270188, 0.43046216,
        0.29031626],
       [0.4180602 , 0.36062262, 0.42013129, 0.38327593, 0.436973  ,
        0.28886535]])

In [ ]:
y_train

array([0., 0., 0., ..., 0., 0., 0.])

## Validation data

In [ ]:
val_loc1 = pd.read_csv('plant2_loc1.csv')
val_loc2 = pd.read_csv('plant2_loc2.csv')
val_loc3 = pd.read_csv('plant2_loc3.csv')


In [ ]:
val_loc1.shape,val_loc2.shape,val_loc3.shape

((57914, 8), (57914, 8), (57914, 8))

In [ ]:
val_loc1.isnull().sum(),val_loc2.isnull().sum(),val_loc3.isnull().sum()

(plant2_train.mea_ddhr           0
 plant2_train.mea_ddhr.1         0
 plant2_train.tem_in_loc1      224
 plant2_train.hum_in_loc1      224
 plant2_train.tem_coil_loc1    224
 plant2_train.tem_out_loc1     224
 plant2_train.hum_out_loc1     224
 plant2_train.cond_loc1        224
 dtype: int64, plant2_train.mea_ddhr           0
 plant2_train.mea_ddhr.1         0
 plant2_train.tem_in_loc2      224
 plant2_train.hum_in_loc2      224
 plant2_train.tem_coil_loc2    224
 plant2_train.tem_out_loc1     224
 plant2_train.hum_out_loc1     224
 plant2_train.cond_loc2        224
 dtype: int64, plant2_train.mea_ddhr           0
 plant2_train.mea_ddhr.1         0
 plant2_train.tem_in_loc3      224
 plant2_train.hum_in_loc3      224
 plant2_train.tem_coil_loc3    224
 plant2_train.tem_out_loc1     224
 plant2_train.hum_out_loc1     224
 plant2_train.cond_loc3        224
 dtype: int64)

In [ ]:
val_loc1.dropna(inplace=True)
val_loc2.dropna(inplace=True)
val_loc3.dropna(inplace=True)


### preprocessing

In [ ]:
from kaggle_study.KMA_Weather import preprocessing_24h48h

prep= preprocessing_24h48h

In [ ]:
plant= '2'
loc1_swv = prep.swv(val_loc1['plant'+plant+'_train.tem_in_loc1'],val_loc1['plant'+plant+'_train.tem_coil_loc1'],val_loc1['plant'+plant+'_train.hum_in_loc1'])
loc2_swv = prep.swv(val_loc2['plant'+plant+'_train.tem_in_loc2'],val_loc2['plant'+plant+'_train.tem_coil_loc2'],val_loc2['plant'+plant+'_train.hum_in_loc2'])
loc3_swv = prep.swv(val_loc3['plant'+plant+'_train.tem_in_loc3'],val_loc3['plant'+plant+'_train.tem_coil_loc3'],val_loc3['plant'+plant+'_train.hum_in_loc3'])

In [ ]:
val_loc1['swv_value'] = loc1_swv
val_loc2['swv_value'] = loc2_swv
val_loc3['swv_value'] = loc3_swv

In [ ]:
loc_num = 'loc1'
mms = MinMaxScaler()

scaled_loc1 = mms.fit_transform(val_loc1[['plant'+plant+'_train.tem_in_'+loc_num, 'plant'+plant+'_train.hum_in_'+loc_num,
       'plant'+plant+'_train.tem_coil_'+loc_num, 'plant'+plant+'_train.tem_out_loc1',
       'plant'+plant+'_train.hum_out_loc1','swv_value']])

scaled_loc2 = mms.fit_transform(val_loc2[['plant'+plant+'_train.tem_in_loc2', 'plant'+plant+'_train.hum_in_loc2',
       'plant'+plant+'_train.tem_coil_loc2', 'plant'+plant+'_train.tem_out_loc1',
       'plant'+plant+'_train.hum_out_loc1','swv_value']])

scaled_loc3 = mms.fit_transform(val_loc3[['plant'+plant+'_train.tem_in_loc3', 'plant'+plant+'_train.hum_in_loc3',
       'plant'+plant+'_train.tem_coil_loc3', 'plant'+plant+'_train.tem_out_loc1',
       'plant'+plant+'_train.hum_out_loc1','swv_value']])

In [ ]:
val_x = np.concatenate([scaled_loc1,scaled_loc2,scaled_loc3])
val_y = np.concatenate([val_loc1['plant2_train.cond_loc1'],val_loc2['plant2_train.cond_loc2'],val_loc3['plant2_train.cond_loc3']])

In [ ]:
val_x.shape,val_y.shape

((173070, 6), (173070,))

In [ ]:
val_x

array([[0.69445089, 0.54678291, 0.83219611, 0.55451551, 0.60833244,
        0.65557991],
       [0.58811238, 0.63494597, 0.82259287, 0.44444444, 0.75838064,
        0.69667713],
       [0.52588809, 0.68479862, 0.81880212, 0.41705668, 0.78526293,
        0.72507524],
       ...,
       [0.35028637, 0.38152556, 0.35317861, 0.32813313, 0.48056121,
        0.19316776],
       [0.35097365, 0.3843657 , 0.35040363, 0.32657306, 0.48548784,
        0.19003554],
       [0.34959908, 0.3909927 , 0.3524218 , 0.32483966, 0.50337367,
        0.1907668 ]])

In [ ]:
val_y

array([0., 0., 0., ..., 0., 0., 0.])

## Test data

In [ ]:
test_loc = pd.read_csv('plant2_test_predict48_temhum_all.csv')
print(test_loc.shape)
test_loc.head()

(1698, 6)


,tem_48h,hum_48h,tem_coil_48h,tem_out_48h,hum_out_48h,swv_value
0,10.844133,44.639767,8.237311,11.486980,43.695072,0.510443
1,12.441169,51.515976,8.432807,12.150341,50.956528,0.361088
2,13.305729,50.456074,8.762803,12.471922,51.867260,0.358948
3,11.406609,45.590240,8.643606,7.784374,57.992565,0.506031
4,11.867703,46.878986,8.621311,8.348910,59.193890,0.467373


In [ ]:
test = pd.read_csv('plant_test_all_predict_final.csv')
test.head()

,Unnamed: 0,plant_test.mea_ddhr,plant_test.plant,plant_test.loc,plant_test.tem_in,plant_test.hum_in,plant_test.tem_coil,plant_test.tem_out_loc1,plant_test.hum_out_loc1,plant_test.x24h_tma,plant_test.x24h_cond_loc,plant_test.x48h_tma,plant_test.x48h_cond_loc
0,1,2019-04-01 0:00,2,3,8.17,40.42,10.10,4.20,54.82,2019-04-02 0:00,0.0,2019-04-03 0:00,0.0
1,2,2019-04-01 4:30,1,2,10.03,48.81,10.79,6.09,59.34,2019-04-02 4:30,NaN,2019-04-03 4:30,NaN
2,3,2019-04-01 10:30,2,3,9.45,40.93,9.07,12.26,32.14,2019-04-02 10:30,0.0,2019-04-03 10:30,0.0
3,4,2019-04-01 11:00,1,1,12.32,34.13,10.86,12.32,28.27,2019-04-02 11:00,NaN,2019-04-03 11:00,NaN
4,5,2019-04-01 13:30,2,1,11.28,22.65,8.76,12.01,19.84,2019-04-02 13:30,0.0,2019-04-03 13:30,0.0


plant 별 데이터 추출

In [ ]:
test_p2 = test[test['plant_test.plant']==2]
test_p2.shape

(1698, 13)

In [ ]:
test_loc.index = pd.to_datetime(test_p2['plant_test.mea_ddhr'])

In [ ]:
test_features = ['tem_48h','hum_48h','tem_coil_48h', 'tem_out_48h', 'hum_out_48h',
       'swv_value']

t_mms = MinMaxScaler()

t_mms.fit(test_loc[test_features])
x_test = t_mms.transform(test_loc[test_features])


In [ ]:
x_test.shape

(1698, 6)

# model

In [ ]:
from sklearn.ensemble import 

In [ ]:
rfc = RandomForestClassifier(n_estimators=500,verbose=1)

In [ ]:
rfc.fit(val_x,val_y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.4min finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=1, warm_start=False)

## val data predict

In [ ]:
val_pred = rfc.predict(val_x)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    4.8s finished


In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(val_y,val_pred)

In [ ]:
acc

0.983492228578032

In [ ]:
val_pred.shape

(173070,)

In [ ]:
val_loc1.columns

Index(['plant2_train.mea_ddhr', 'plant2_train.mea_ddhr.1',
       'plant2_train.tem_in_loc1', 'plant2_train.hum_in_loc1',
       'plant2_train.tem_coil_loc1', 'plant2_train.tem_out_loc1',
       'plant2_train.hum_out_loc1', 'plant2_train.cond_loc1', 'swv_value'],
      dtype='object')

In [ ]:
val_loc1.rename({'plant2_train.tem_in_loc1':'tem_in', 'plant2_train.hum_in_loc1':'hum_in',
       'plant2_train.tem_coil_loc1':'tem_coil', 'plant2_train.tem_out_loc1':'tem_out',
       'plant2_train.hum_out_loc1':'hum_out', 'plant2_train.cond_loc1':'cond'},axis=1,inplace=True)

val_loc2.rename({'plant2_train.tem_in_loc2':'tem_in', 'plant2_train.hum_in_loc2':'hum_in',
       'plant2_train.tem_coil_loc2':'tem_coil', 'plant2_train.tem_out_loc1':'tem_out',
       'plant2_train.hum_out_loc1':'hum_out', 'plant2_train.cond_loc2':'cond'},axis=1,inplace=True)

val_loc3.rename({'plant2_train.tem_in_loc3':'tem_in', 'plant2_train.hum_in_loc3':'hum_in',
       'plant2_train.tem_coil_loc3':'tem_coil', 'plant2_train.tem_out_loc1':'tem_out',
       'plant2_train.hum_out_loc1':'hum_out', 'plant2_train.cond_loc3':'cond'},axis=1,inplace=True)

In [ ]:
val_loc = pd.concat([val_loc1,val_loc2,val_loc3])

In [ ]:
val_loc['pred_y'] = val_pred

In [ ]:
val_loc.shape

(173070, 10)

In [ ]:
val_loc[(val_loc['cond']==1)]

,plant2_train.mea_ddhr,plant2_train.mea_ddhr.1,tem_in,hum_in,tem_coil,tem_out,hum_out,cond,swv_value,pred_y
1209,2016-12-21 21:00:00,2016-12-21 21:00:00,11.43,84.34,9.18,10.61,99.27,1.0,0.022920,1.0
1210,2016-12-22 00:00:00,2016-12-22 00:00:00,11.65,88.86,9.45,12.29,99.27,1.0,-0.034417,1.0
1211,2016-12-22 03:00:00,2016-12-22 03:00:00,12.56,92.37,9.87,13.36,99.27,1.0,-0.126894,1.0
1995,2017-01-26 23:00:00,2017-01-26 23:00:00,5.36,71.83,1.32,5.39,77.93,1.0,0.029544,1.0
1996,2017-01-27 00:00:00,2017-01-27 00:00:00,5.30,72.89,1.74,4.93,79.95,1.0,0.043373,1.0
...,...,...,...,...,...,...,...,...,...,...
41024,2018-12-03 17:20:00,2018-12-03 17:20:00,15.34,81.78,12.58,14.73,99.33,1.0,0.031678,1.0
41025,2018-12-03 17:30:00,2018-12-03 17:30:00,15.31,81.26,12.96,14.61,99.30,1.0,0.080223,1.0
41026,2018-12-03 17:40:00,2018-12-03 17:40:00,15.40,81.01,13.00,14.55,99.27,1.0,0.080320,1.0
41028,2018-12-03 18:00:00,2018-12-03 18:00:00,15.34,81.17,12.58,14.37,99.27,1.0,0.042314,1.0


In [ ]:
len(val_loc[(val_loc['cond']==1) & (val_loc['pred_y']==1)])

572

## Test data predict

In [ ]:
test_pred= rfc.predict_proba(x_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.1s finished


In [ ]:
test_pred.shape

(1698, 2)

In [ ]:
for i in test_pred[:,1]:
    print(i)

In [ ]:
test_p2['48h_prob']= test_pred[:,1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
test_p2

,Unnamed: 0,plant_test.mea_ddhr,plant_test.plant,plant_test.loc,plant_test.tem_in,plant_test.hum_in,plant_test.tem_coil,plant_test.tem_out_loc1,plant_test.hum_out_loc1,plant_test.x24h_tma,plant_test.x24h_cond_loc,plant_test.x48h_tma,plant_test.x48h_cond_loc,24h_prob,48h_prob
plant_test.mea_ddhr,,,,,,,,,,,,,,,
2019-04-01 00:00:00,1,2019-04-01 0:00,2,3,8.17,40.42,10.10,4.20,54.82,2019-04-02 0:00,0.0,2019-04-03 0:00,0.0,0.0,0.000
2019-04-01 10:30:00,3,2019-04-01 10:30,2,3,9.45,40.93,9.07,12.26,32.14,2019-04-02 10:30,0.0,2019-04-03 10:30,0.0,0.0,0.002
2019-04-01 13:30:00,5,2019-04-01 13:30,2,1,11.28,22.65,8.76,12.01,19.84,2019-04-02 13:30,0.0,2019-04-03 13:30,0.0,0.0,0.002
2019-04-01 17:00:00,6,2019-04-01 17:00,2,2,12.26,25.46,9.45,9.66,29.52,2019-04-02 17:00,0.0,2019-04-03 17:00,0.0,0.0,0.000
2019-04-01 18:30:00,7,2019-04-01 18:30,2,3,11.62,25.55,9.45,7.99,31.81,2019-04-02 18:30,0.0,2019-04-03 18:30,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-27 19:00:00,3524,2020-03-27 19:00,2,3,14.06,36.32,15.06,8.35,51.47,2020-03-28 19:00,0.0,2020-03-29 19:00,0.0,0.0,0.000
2020-03-28 14:30:00,3531,2020-03-28 14:30,2,2,13.48,23.14,13.08,9.88,29.70,2020-03-29 14:30,0.0,2020-03-30 14:30,0.0,0.0,0.000
2020-03-28 23:00:00,3532,2020-03-28 23:00,2,1,10.55,34.25,11.17,7.04,49.11,2020-03-29 23:00,0.0,2020-03-30 23:00,0.0,0.0,0.000


In [ ]:
test.index = pd.to_datetime(test['plant_test.mea_ddhr'])

In [ ]:
test.loc[(test['plant_test.plant']==1) ,'plant_test.x48h_cond_loc'] = test_p2['plant_test.x48h_cond_loc'].values

In [ ]:
test

In [ ]:
test.loc[(test['plant_test.plant']==1),'plant_test.x48h_cond_loc'].isnull().sum()

0

In [ ]:
test.to_csv('plant_test_all_predict_final.csv',index=False)

## 검증 데이터셋
- 분류뿐만 아니라 확률값도 기입

In [ ]:
val_set = pd.read_csv('203611.csv')
print(val_set.shape)
val_set.head()

(3539, 9)


,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
0,2019-04-01 0:00,2,3,2019-04-02 0:00,NaN,NaN,2019-04-03 0:00,NaN,NaN
1,2019-04-01 4:30,1,2,2019-04-02 4:30,NaN,NaN,2019-04-03 4:30,NaN,NaN
2,2019-04-01 10:30,2,3,2019-04-02 10:30,NaN,NaN,2019-04-03 10:30,NaN,NaN
3,2019-04-01 11:00,1,1,2019-04-02 11:00,NaN,NaN,2019-04-03 11:00,NaN,NaN
4,2019-04-01 13:30,2,1,2019-04-02 13:30,NaN,NaN,2019-04-03 13:30,NaN,NaN


In [ ]:
val_set.index = pd.to_datetime(val_set['MEA_DDHR'])

In [ ]:
val_set.loc[val_set['PLANT']==2]

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
MEA_DDHR,,,,,,,,,
2019-04-01 00:00:00,2019-04-01 0:00,2,3,2019-04-02 0:00,0.0,NaN,2019-04-03 0:00,0.0,NaN
2019-04-01 10:30:00,2019-04-01 10:30,2,3,2019-04-02 10:30,0.0,NaN,2019-04-03 10:30,0.0,NaN
2019-04-01 13:30:00,2019-04-01 13:30,2,1,2019-04-02 13:30,0.0,NaN,2019-04-03 13:30,0.0,NaN
2019-04-01 17:00:00,2019-04-01 17:00,2,2,2019-04-02 17:00,0.0,NaN,2019-04-03 17:00,0.0,NaN
2019-04-01 18:30:00,2019-04-01 18:30,2,3,2019-04-02 18:30,0.0,NaN,2019-04-03 18:30,0.0,NaN
...,...,...,...,...,...,...,...,...,...
2020-03-27 19:00:00,2020-03-27 19:00,2,3,2020-03-28 19:00,0.0,NaN,2020-03-29 19:00,0.0,NaN
2020-03-28 14:30:00,2020-03-28 14:30,2,2,2020-03-29 14:30,0.0,NaN,2020-03-30 14:30,0.0,NaN
2020-03-28 23:00:00,2020-03-28 23:00,2,1,2020-03-29 23:00,0.0,NaN,2020-03-30 23:00,0.0,NaN


In [ ]:
val_set.loc[val_set['PLANT']==2,'X48H_COND_LOC_PROB'] = test_p2['48h_prob'].values

In [ ]:
val_set

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
MEA_DDHR,,,,,,,,,
2019-04-01 00:00:00,2019-04-01 0:00,2,3,2019-04-02 0:00,0.0,0.0,2019-04-03 0:00,0.0,0.000
2019-04-01 04:30:00,2019-04-01 4:30,1,2,2019-04-02 4:30,0.0,0.0,2019-04-03 4:30,0.0,0.000
2019-04-01 10:30:00,2019-04-01 10:30,2,3,2019-04-02 10:30,0.0,0.0,2019-04-03 10:30,0.0,0.002
2019-04-01 11:00:00,2019-04-01 11:00,1,1,2019-04-02 11:00,0.0,0.0,2019-04-03 11:00,0.0,0.000
2019-04-01 13:30:00,2019-04-01 13:30,2,1,2019-04-02 13:30,0.0,0.0,2019-04-03 13:30,0.0,0.002
...,...,...,...,...,...,...,...,...,...
2020-03-29 03:00:00,2020-03-29 3:00,1,2,2020-03-30 3:00,0.0,0.0,2020-03-31 3:00,0.0,0.008
2020-03-29 05:00:00,2020-03-29 5:00,2,3,2020-03-30 5:00,0.0,0.0,2020-03-31 5:00,0.0,0.000
2020-03-29 07:00:00,2020-03-29 7:00,2,1,2020-03-30 7:00,0.0,0.0,2020-03-31 7:00,0.0,0.000


In [ ]:
val_set['X48H_COND_LOC_PROB'].isnull().sum()

0

In [ ]:
val_set.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3539 entries, 2019-04-01 00:00:00 to 2020-03-29 19:30:00
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   MEA_DDHR            3539 non-null   object 
 1   PLANT               3539 non-null   int64  
 2   LOC                 3539 non-null   int64  
 3   X24H_TMA            3539 non-null   object 
 4   X24H_COND_LOC       3539 non-null   float64
 5   X24H_COND_LOC_PROB  1841 non-null   float64
 6   X48H_TMA            3539 non-null   object 
 7   X48H_COND_LOC       3539 non-null   float64
 8   X48H_COND_LOC_PROB  1841 non-null   float64
dtypes: float64(4), int64(2), object(3)
memory usage: 276.5+ KB


In [ ]:
val_set.to_csv('203611.csv',index=False)

In [ ]:
val_set2 = pd.read_csv('203611.csv')
val_set2

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
0,2019-04-01 0:00,2,3,2019-04-02 0:00,0.0,0.0,2019-04-03 0:00,0.0,0.0
1,2019-04-01 4:30,1,2,2019-04-02 4:30,0.0,0.0,2019-04-03 4:30,0.0,0.0
2,2019-04-01 10:30,2,3,2019-04-02 10:30,0.0,0.0,2019-04-03 10:30,0.0,0.2
3,2019-04-01 11:00,1,1,2019-04-02 11:00,0.0,0.0,2019-04-03 11:00,0.0,0.0
4,2019-04-01 13:30,2,1,2019-04-02 13:30,0.0,0.0,2019-04-03 13:30,0.0,0.2
...,...,...,...,...,...,...,...,...,...
3534,2020-03-29 3:00,1,2,2020-03-30 3:00,0.0,0.0,2020-03-31 3:00,0.0,0.8
3535,2020-03-29 5:00,2,3,2020-03-30 5:00,0.0,0.0,2020-03-31 5:00,0.0,0.0
3536,2020-03-29 7:00,2,1,2020-03-30 7:00,0.0,0.0,2020-03-31 7:00,0.0,0.0
3537,2020-03-29 11:30,1,1,2020-03-30 11:30,0.0,0.0,2020-03-31 11:30,0.0,0.0


In [ ]:
a= val_set2['X24H_COND_LOC_PROB']*100

In [ ]:
val_set2['X24H_COND_LOC_PROB'] = a

In [ ]:
val_set2['X48H_COND_LOC_PROB'] = val_set2['X48H_COND_LOC_PROB'].astype(int).values

In [ ]:
val_set2['X48H_COND_LOC'] = val_set2['X48H_COND_LOC'].astype(int).values

In [ ]:
val_set2[val_set2['X24H_COND_LOC_PROB'] >=70]

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
1005,2019-07-20 10:00,1,3,2019-07-21 10:00,1,90,2019-07-22 10:00,1,90
1162,2019-08-06 9:00,1,3,2019-08-07 9:00,1,86,2019-08-08 9:00,0,0
1495,2019-09-16 2:00,1,1,2019-09-17 2:00,1,90,2019-09-18 2:00,0,0


In [ ]:
val_set2.to_csv('203611.csv',index=False)